# Proyecto individual: Sistema de recomendación de películas
​
Este proyecto constará de dos fases: `Ingenieria de datos`, `Modelamiento y evaluación con machine learning`.

## 1. Ingeniería de datos
### 1.1. Transformaciones de datos
### 1.2. Análisis exploratorio de datos o EDA
## 2. Modelamiento y evaluación con machine learning
* Implementar un modelo de clasificación con aprendizaje supervisado que permita clasificar (**con un algoritmo de similitud de coseno**) las 5 películas más similares considerando estas variables:
    * release_year,
    * budget,
    * popularity,
    * vote_average,
    * genres_name,
    * studios_name.               

## 2. Modelamiento y evaluación

#### 2.1. Consideraciones
* 2.1.1. Se procederá a probar los tiempos de ejecución con distintas variables, primero numéricas, y luego se agregarán las categóricas. Se buscará que el algoritmo en esta computadora se demore menos a 5 segundos.
* 2.1.2. Se opta por trabajar solo con 6 variables porque por experiencia es un buen número. Esto quiere decir que se descartan columnas que se trabajaron en el eda, como ser:
    * revenue,
    * return,
    * vote_count.
* 2.1.3. Se podría haber realizado un feature importance o una matriz de correlación, pero el trabajo entre variable cualitativas y numéricas es pesado y no tiene sentido para este MVP.

#### 2.2. Importación de la librería

In [21]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer

#### 2.3. Carga y visualización del DataFrame

In [22]:
df_ml_filter = pd.read_parquet('dataset/data_movies_ml.parquet')
df_ml_filter.head()

,budget,popularity,revenue,title,vote_average,vote_count,release_year,studios_name,genres_name,return
0,30000000.0,21.95,373554033.0,Toy Story,7.7,5415.0,1995,['Pixar Animation Studios'],"['Animation', 'Comedy', 'Family']",12.451801
1,65000000.0,17.02,262797249.0,Jumanji,6.9,2413.0,1995,"['TriStar Pictures', 'Teitler Film', 'Intersco...","['Adventure', 'Fantasy', 'Family']",4.043035
2,16000000.0,3.86,81452156.0,Waiting to Exhale,6.1,34.0,1995,['Twentieth Century Fox Film Corporation'],"['Comedy', 'Drama', 'Romance']",5.090760
3,60000000.0,17.92,187436818.0,Heat,7.7,1886.0,1995,"['Regency Enterprises', 'Forward Pass', 'Warne...","['Action', 'Crime', 'Drama', 'Thriller']",3.123947
4,35000000.0,5.23,64350171.0,Sudden Death,5.5,174.0,1995,"['Universal Pictures', 'Imperial Entertainment...","['Action', 'Adventure', 'Thriller']",1.838576


#### 2.4. Modelo de prueba con las siguientes variables (numéricas):
* 'budget', 
* 'vote_average', 
* 'popularity',
* 'release_year'

In [23]:
# Primero, vamos a convertir la columna 'title' a una representación numérica usando TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_ml_filter['title'])

# Luego, vamos a añadir las columnas numéricas a nuestra matriz de características
features = np.column_stack([tfidf_matrix.toarray(),
                            df_ml_filter['budget'], 
                            df_ml_filter['vote_average'], 
                            df_ml_filter['popularity'],
                            df_ml_filter['release_year']])

# Reindexamos el DataFrame
data = df_ml_filter.reset_index(drop=True)

# Ahora, calculamos la matriz de similitud de coseno
similarity_matrix = cosine_similarity(features)

# Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
name_title = "Toy Story"  # El nombre del producto para el que quieres hacer recomendaciones
name_films = data[data['title'] == name_title]

if not name_films.empty:
    product_index = name_films.index[0]
    product_similarities = similarity_matrix[product_index]
    
    # Se obtienen los índices de las 5 películas más similares (ordenados de forma descendente), excluyendo la propia película
    most_similar_films_indices = np.argsort(-product_similarities)[1:6]
    most_similar_films = data.loc[most_similar_films_indices, ['title', 'vote_average', 'popularity', 'release_year']]
    print(f"Las 5 películas más recomendadas para {name_title} son:")
    print(most_similar_films.to_string(index=False))
else:
    print("Película no encontrada")


Las 5 películas más recomendadas para Toy Story son:
                   title  vote_average  popularity  release_year
                    Babe           6.0       14.40          1995
The Hunt for Red October           7.2       15.91          1990
       Kill Bill: Vol. 1           7.7       25.26          2003
                Identity           7.1       17.96          2003
       Kill Bill: Vol. 2           7.7       21.53          2004


#### 2.5. Modelo de prueba con las siguientes variables (numéricas y categóricas):
* 'budget', 
* 'vote_average', 
* 'popularity',
* 'release_year',
* 'genres_name

In [24]:
# Vectorizar la columna 'title' usando TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_ml_filter['title'])

# Codificar la columna 'genres_name' usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df_ml_filter['genres_name'])

# Luego, vamos a añadir las columnas numéricas y la matriz de géneros a nuestra matriz de características
features = np.column_stack([tfidf_matrix.toarray(),
                            df_ml_filter['budget'], 
                            df_ml_filter['vote_average'], 
                            df_ml_filter['popularity'],
                            df_ml_filter['release_year'],
                            genres_encoded])

# Reindexamos el DataFrame
data = df_ml_filter.reset_index(drop=True)

# Ahora, calculamos la matriz de similitud de coseno
similarity_matrix = cosine_similarity(features)

# Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
name_title = "Toy Story"  # El nombre de la película para la cual quieres hacer recomendaciones
name_films = data[data['title'] == name_title]

if not name_films.empty:
    product_index = name_films.index[0]
    product_similarities = similarity_matrix[product_index]
    
    # Se obtienen los índices de las 5 películas más similares (ordenados de forma descendente), excluyendo la propia película
    most_similar_products_indices = np.argsort(-product_similarities)[1:6]
    
    # Obtener los títulos, vote_average y release_year de las 5 películas más similares
    most_similar_products = data.loc[most_similar_products_indices, ['title', 'vote_average', 'genres_name', 'release_year']]
    
    print(f"Las 5 películas más recomendadas para {name_title} son:")
    print(most_similar_products.to_string(index=False))
else:
    print("Película no encontrada")


Las 5 películas más recomendadas para Toy Story son:
                   title  vote_average                                 genres_name  release_year
The Hunt for Red October           7.2                        ['Action', 'Comedy']          1990
                    Babe           6.0 ['Adventure', 'History', 'Drama', 'Family']          1995
       Kill Bill: Vol. 1           7.7                                  ['Comedy']          2003
                Identity           7.1                ['Action', 'Drama', 'Crime']          2003
       Kill Bill: Vol. 2           7.7                        ['Comedy', 'Horror']          2004


#### 2.6. Modelo de prueba con las siguientes variables (numéricas y categóricas):
* 'budget', 
* 'vote_average', 
* 'popularity',
* 'release_year',
* 'genres_name',
* 'studios_name'.

In [25]:
# Vectorizar la columna 'title' usando TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_ml_filter['title'])

# Codificar la columna 'genres_name' y 'studios_name' usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df_ml_filter['genres_name'])
studios_encoded = mlb.fit_transform(df_ml_filter['studios_name'])

# Luego, vamos a añadir las columnas numéricas y la matriz de géneros a nuestra matriz de características
features = np.column_stack([tfidf_matrix.toarray(),
                            df_ml_filter['budget'], 
                            df_ml_filter['vote_average'], 
                            df_ml_filter['popularity'],
                            df_ml_filter['release_year'],
                            genres_encoded,
                            studios_encoded])

# Reindexamos el DataFrame
data = df_ml_filter.reset_index(drop=True)

# Ahora, calculamos la matriz de similitud de coseno
similarity_matrix = cosine_similarity(features)

# Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
name_title = "Toy Story"  # El nombre de la película para la cual quieres hacer recomendaciones
name_films = data[data['title'] == name_title]

if not name_films.empty:
    product_index = name_films.index[0]
    product_similarities = similarity_matrix[product_index]
    
    # Se obtienen los índices de las 5 películas más similares (ordenados de forma descendente), excluyendo la propia película
    most_similar_products_indices = np.argsort(-product_similarities)[1:6]
    
    # Obtener los títulos, vote_average y release_year de las 5 películas más similares
    most_similar_products = data.loc[most_similar_products_indices, ['title', 'vote_average', 'genres_name', 'studios_name', 'release_year']]
    
    print(f"Las 5 películas más recomendadas para {name_title} son:")
    print(most_similar_products.to_string(index=False))
else:
    print("Película no encontrada")


Las 5 películas más recomendadas para Toy Story son:
                   title  vote_average                                 genres_name                                           studios_name  release_year
The Hunt for Red October           7.2                        ['Action', 'Comedy'] ['Weintraub Entertainment Group', 'Elrina Investment']          1990
                    Babe           6.0 ['Adventure', 'History', 'Drama', 'Family']                              ['Sony New Technologies']          1995
       Kill Bill: Vol. 2           7.7                        ['Comedy', 'Horror']                                 ['ArieScope Pictures']          2004
       Kill Bill: Vol. 1           7.7                                  ['Comedy']                          ['Metro-Goldwyn-Mayer (MGM)']          2003
                Identity           7.1                ['Action', 'Drama', 'Crime']                                                     []          2003


#### 2.7. Observaciones
* Se decide que, dado que algoritmo con las 4 variables numéricas y 2 categóricas corre en menos de 5 segundos, se decide formular la función para el endpoint con este modelo. 

#### 2.8. Función para el endpoint de recomendación:

In [26]:
def recomendacion(df, titulo):
    # Vectorizar la columna 'title' usando TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['title'])

    # Codificar la columna 'genres_name' y 'studios_name' usando MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    genres_encoded = mlb.fit_transform(df['genres_name'])
    studios_encoded = mlb.fit_transform(df['studios_name'])

    # Luego, vamos a añadir las columnas numéricas y la matriz de géneros a nuestra matriz de características
    features = np.column_stack([tfidf_matrix.toarray(),
                                df['budget'], 
                                df['vote_average'], 
                                df['popularity'],
                                df['release_year'],
                                genres_encoded,
                                studios_encoded])

    # Reindexamos el DataFrame
    data = df.reset_index(drop=True)

    # Ahora, calculamos la matriz de similitud de coseno
    similarity_matrix = cosine_similarity(features)

    # Para hacer recomendaciones, puedes buscar las películas más similares a una película dada
    name_films = data[data['title'] == titulo]

    if not name_films.empty:
        product_index = name_films.index[0]
        product_similarities = similarity_matrix[product_index]

        # Se obtienen los índices de las 5 películas más similares (ordenados de forma descendente), excluyendo la propia película
        most_similar_products_indices = np.argsort(-product_similarities)[1:6]

        # Obtener los títulos de las 5 películas más similares
        most_similar_products_titles = data.loc[most_similar_products_indices, 'title'].tolist()

        return {f'Las 5 películas más similares a {titulo} son': most_similar_products_titles}
    else:
        return "Película no encontrada"

In [27]:
recomendacion(df_ml_filter, 'Toy Story')

{'Las 5 películas más similares a Toy Story son': ['The Hunt for Red October',
  'Babe',
  'Kill Bill: Vol. 2',
  'Kill Bill: Vol. 1',
  'Identity']}